<h1><center>Case Study 12: Neural Networks</center></h1>
<h3><center>Steven Cocke, Hannah Kosinovsky, Tanvi Arora</center></h3>
<h3><center>November 18th, 2019</center></h3>

<h3><center>Abstract</center></h3>

In this case study we attempted to replicate the work in the paper on "Searching for Exotic Particles in High-Energy Physics with Deep Learning" by adjusting the parameters and hyper parameters in keras to build a deep learning model. 

## 1 Introduction

Neural Networks are statistical models that were named after brain neurons. Neurons in the brain fires after a certain threshold is reached. In the statistical model, after the inputs of the model reach a certain threshold, the neuron in the model also "fires". Since we have multiple neurons in the model, it becomes a network. Each neuron in our model computes an "activation function".

These activation functions act as logistic regressors. Since the layers in the network perform these functions several times over on each previous output, the neural networks are essentially doing regressions on top of one another. 

The goal is to change the inputs such that the model has more complex indpendent components. Each neuron in the network has its own weight and bias. 

In the paper from which we are trying to replicate results, *I need help describing the performance results from the paper itself, not sure which values are most relevent I'm sorry*

By following the methodology for the inputs, we should be able to obtain similar accuracy. 

## 2 Methods

In order to replicate these results, we had to focus on several different inputs. In the paper it was specified that they used 

- tanh activation functions in the hidden layers

<a id='tanhmethods'></a>
<a href="#tanh">Tanh Activation Function</a>

- weights initialized from a normal distribution with zero mean and standard deviation of .1. 

<a id='weightmethods'></a>
<a href="#weights">Weight Initialization</a>

- gradient computations made on mini-batches of size 100

<a id='minibatchmethods'></a>
<a href="#minibatch">Mini-Batches</a>

- momentum that increases linearly between .9 and .99

<a id='momentummethods'></a>
<a href="#momentum">Momentum Adjusting</a>

- Implementation of early stopping after a set threshold to prevent overfitting

<a id='stoppingmethods'></a>
<a href="#stopping">Early Stopping</a>


## 3 Results

*compare our final results to the accuracy rate that they had. 


## 4 Conclusions

We were not able to obtain an accuracy rate as high as the one in the paper. It is possible that there were additional inputs not accurately specified. 

Overall it is no longer good practice to use Stochastic Gradient Descent (SGD) when using tensorflow to produce neural networks. Now, it has been shown that the optimizer "Adam" outperforms SGD, which is shown to be better for shallow networks. In order to improve future results we would suggest using Adam as the optimizer. 

## References

Paper on Deep Neural Networks used to replicate results: 
https://arxiv.org/pdf/1402.4735.pdf

Comparison of keras optimizers:
https://www.dlology.com/blog/quick-notes-on-how-to-choose-optimizer-in-keras/

## Code

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.regularizers import l2
from sklearn import datasets
import pandas as pd
import numpy as np
print(tf.__version__)

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
higgs=pd.read_csv('HIGGS.csv',sep=',',nrows=2.6E6,header=None,dtype=np.float16)
#higgs=pd.read_csv('HIGGS.csv',sep=',',header=None,dtype=np.float16)

In [ ]:
higgs.head()

In [ ]:
higgs.columns=['class_label','lepton_pT','lepton_eta','lepton_phi','missing_energy_magnitude','missing_energy_phi','jet_1_pt','jet_1_eta','jet_1_phi','jet_1_b_tag','jet_2_pt','jet_2_eta','jet_2_phi','jet_2_b_tag','jet_3_pt','jet_3_eta','jet_3_phi','jet_3_b_tag','jet_4_pt','jet_4_eta','jet_4_phi','jet_4_b_tag','m_jj','m_jjj','m_lv','m_jlv','m_bb','m_wbb','m_wwbb']

In [ ]:
higgs.head()

In [ ]:
higgs.describe()

In [ ]:
higgs.isnull().sum()

In [ ]:
x=higgs.copy()
x=x.drop(['class_label'],axis=1)
x.head()

In [ ]:
y=higgs['class_label']
y.head()

### Scale Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_train = scaler.fit_transform(x)

# Print out the adjustment that the scaler applied to the total_earnings column of data
print("Note: median values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[12], scaler.min_[12]))
multiplied_by = scaler.scale_[12]
added = scaler.min_[12]

scaled_train_df = pd.DataFrame(scaled_train, columns=x.columns.values)

In [ ]:
for i in scaled_train_df:
    scaled_train_df[i].hist()
    plt.show()

### Model

In [ ]:
x.shape

<a id='tanh'></a>
<a href="#tanhmethods">Back to Top</a>

In [ ]:
model = tf.keras.Sequential()
model.add(layers.Dense(300, activation='tanh'))
model.add(layers.Dense(300, activation='tanh'))
model.add(layers.Dense(300, activation='tanh'))
model.add(layers.Dense(300, activation='tanh'))
model.add(layers.Dense(1,activation='tanh'))

### Compile Model

In [ ]:


learning_rate=0.05
decay_rate=1E-5
sgd = SGD(lr=learning_rate,  decay=decay_rate, nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

#model.compile(optimizer='sgd',loss='mean_squared_error',metrics=['mean_squared_error'])

### Fit Model

In [ ]:
print(type(x))
print(type(y))

y= np.asarray(y)
print(type(y))

In [ ]:
model.fit(scaled_train_df.values, y, epochs=5, batch_size=32)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(scaled_train_df, y, test_size=0.38, random_state=1776)

In [ ]:
print(x_test.shape)
print(type(np.asarray(y_test)).shape)

In [ ]:
# Evaluate the model on the test data using `evaluate`
print('\n# Evaluate on test data')
results = model.evaluate(np.asarray(x_test), np.asarray(y_test), batch_size=128)
print('test loss, test acc:', results)

In [ ]:
# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
#print('\n# Generate predictions for 3 samples')
predictions = model.predict(np.asarray(x))
print('predictions shape:', predictions.shape)

In [ ]:
from sklearn import metrics

fpr_keras, tpr_keras, thresholds_keras = metrics.roc_curve(y,predictions,pos_label=None)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()


In [ ]:
print(auc_keras)

### Hyperparameters

hidden layers - tanh function  

initial weights - normal distribution with zero mean and std = 0.1 in first layer , 0.001 in the output layer and 0.05 in all other hidden layers

gradient computations on batch of size = 100


momentum - increased linearly over first 200 epochs from 0.9 to 0.99 , after which it is constant  

learning rate decayed by a factor of 1.0000002 every batch update until it reached a minimum of 10 power -6

Training ended when momentum reached maximum value and min error on validation set had not decreased by more than a factor of 0.00001 over 10 epochs



<a id='weights'></a>
<a href="#weightmethods">Back to Top</a>

In [ ]:
o_w1=model.layers[1].get_weights()
print(o_w1[0].shape)
print(o_w1[1].shape)

In [ ]:
type(np.random.normal(0, 0.1, 300))

In [ ]:

w1= [np.random.normal(0, 0.1, 300),np.asarray([0]*300)]
model.layers[1].set_weights(w1)
#print(type(w1))

In [ ]:
w1= [np.random.normal(0, 0.1, 300),[0]*300]
model.layers[1].set_weights(w1)
w2= np.random.normal(0, 0.05, 300)
model.layers[2].set_weights(w2)
w3= np.random.normal(0, 0.05, 300)
model.layers[3].set_weights(w3)
w4= np.random.normal(0, 0.05, 300)
model.layers[4].set_weights(w4)
w5= np.random.normal(0, 0.001, 1)
model.layers[5].set_weights(w5)

<a id='minibatch'></a>
<a href="#minibatchmethods">Back to Top</a>

In [ ]:
def step_decay(epoch):
    initial_lrate = 0.05
    lrate=initial_lrate/((1+epoch)*1.0000002)
    if lrate>=10E-6:
        return lrate
    else:
        return None


<a id='stopping'></a>
<a href="#stoppingmethods">Back to Top</a>

In [ ]:
# learning schedule callback
lrate = LearningRateScheduler(step_decay)
earlystopper= EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=10)
callbacks_list = [lrate,earlystopper]
# Fit the model
model.fit(scaled_train_df.values, y, validation_split=0.3, epochs=40, batch_size=100, callbacks=callbacks_list, verbose=2)

<a id='momentum'></a>
<a href="#momentummethods">Back to Top</a>

In [ ]:
def get_momentum(epoch):
    return min(epoch*(0.09/200),0.09)

In [ ]:
predictions2 = model.predict(np.asarray(x))
fpr_keras2, tpr_keras2, thresholds_keras2 = metrics.roc_curve(y,predictions2,pos_label=None)
auc_keras2 = auc(fpr_keras2, tpr_keras2)

In [ ]:
plt.figure(1)
plt.plot([0, 1], 'k--')
plt.plot(fpr_keras2, tpr_keras2, label='Keras (area = {:.3f})'.format(auc_keras2))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
sgd = SGD(lr=learning_rate,  decay=decay_rate,nesterov=False)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model2 = tf.keras.Sequential()
model2.add(layers.Dense(300, activation='tanh',kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.1, seed=None)))
model2.add(layers.Dense(300, activation='tanh',kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
model2.add(layers.Dense(300, activation='tanh',kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
model2.add(layers.Dense(300, activation='tanh',kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
model2.add(layers.Dense(1,activation='tanh',kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.001, seed=None)))

In [ ]:
learning_rate=0.05
decay_rate=1E-5
sgd = SGD(lr=learning_rate,  decay=decay_rate, momentum=0.09,nesterov=False)
model2.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
model2.fit(scaled_train_df.values, y, epochs=5, batch_size=32)

In [ ]:
# learning schedule callback
lrate = LearningRateScheduler(step_decay)
earlystopper= EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=10)
callbacks_list = [lrate,earlystopper]
# Fit the model
model2.fit(scaled_train_df.values, y, validation_split=0.3, epochs=40, batch_size=100, callbacks=callbacks_list, verbose=2)

In [ ]:
predictions2 = model2.predict(np.asarray(scaled_train_df.values))
fpr_keras2, tpr_keras2, thresholds_keras2 = metrics.roc_curve(y,predictions2,pos_label=None)
auc_keras2 = auc(fpr_keras2, tpr_keras2)
print(auc_keras2)

In [ ]:
plt.figure(1)
plt.plot([0, 1], 'k--')
plt.plot(fpr_keras2, tpr_keras2, label='Keras (area = {:.3f})'.format(auc_keras2))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()